#### 第六章 使用一等函数实现设计模式
##### 6.1 案例分析：重构“策略”模式

In [4]:
# 实现Order类， 支持插入式折扣策略
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [5]:
class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

In [6]:
class Order: # 上下文
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [7]:
class Promotion(ABC): # 策略：抽象基类
    
    @abstractmethod
    def discount(self, order):
        '''返回折扣金额（正值）'''

In [8]:
class FidelityPromo(Promotion): # 第一个具体策略
    '''为积分为1000或以上的顾客提供5%折扣'''
    
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [9]:
class BulkItemPromo(Promotion): # 第二个具体策略
    '''单个商品为20个或以上是提供10%折扣'''
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

In [10]:
class LargeOrderPromo(Promotion): # 第三个具体策略
    '''订单中的不同商品达到10个或以上时提供7%折扣'''
    
    def discount(self, order):
        distinc_items = {item.product for item in order.cart}
        if len(distinc_items) >= 10:
            return order.total() * .07
        return 0

In [19]:
# 使用不同促销折扣的Order类示例
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
o_1 = Order(joe, cart, FidelityPromo())
print(o_1)
o_2 = Order(ann, cart, FidelityPromo())
print(o_2)
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
o_3 = Order(joe, banana_cart, BulkItemPromo())
print(o_3)
long_order = [LineItem(str(item_code), 1, 1.0)
             for item_code in range(10)]
o_4 = Order(joe, long_order, LargeOrderPromo())
print(o_4)
o_5 = Order(joe, cart, LargeOrderPromo())
print(o_5)

<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 39.90>
<Order total: 30.00 due: 28.50>
<Order total: 10.00 due: 9.30>
<Order total: 42.00 due: 42.00>


In [22]:
# Order类和使用函数实现的折扣策略
class Order1:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [24]:
def fidelity_promo(order): 
    """为积分为1000或以上的顾客提供5%折扣"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [26]:
def bulk_item_promo(order):
    """单个商品为20个或以上时提供10%折扣"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

In [27]:
def large_order_promo(order):
    """订单中的不同商品达到10个或以上时提供7%折扣"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [28]:
# best_promo 迭代一个函数列表，并找出折扣额度最大的
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    '''选择可用的最佳折扣'''
    
    return max(promo(order) for promo in promos)

In [30]:
print(Order1(joe, long_order, best_promo))
print(Order1(joe, banana_cart, best_promo))
print(Order1(ann, cart, best_promo))

<Order total: 10.00 due: 9.30>
<Order total: 30.00 due: 28.50>
<Order total: 42.00 due: 39.90>


In [40]:
# 内省模块的全局命名空间， 构建promos列表
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']
print(promos)

def best_prom(order):
    '''选择可用的最佳折扣'''
    
    return max(promo(order) for promo in promos)

[<function fidelity_promo at 0x0000023C2CB1B840>, <function bulk_item_promo at 0x0000023C2CBB7488>, <function large_order_promo at 0x0000023C2CBB7598>]


##### 6.2 “命令”模式


#### 第七章 函数装饰器和闭包
函数装饰器用于在源码中“标记”函数，以某种方式增强函数的行为。这
是一项强大的功能，但是若想掌握，必须理解闭包。
##### 7.1 装饰器基础知识
装饰器是可调用的对象，其参数是另一个函数（被装饰的函数）。 装
饰器可能会处理被装饰的函数，然后把它返回，或者将其替换成另一个
函数或可调用对象。

In [6]:
from functools import wraps
def deco(func):
    @wraps(func)
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [7]:
target()
print(target)

running inner()
<function target at 0x00000229A925AC80>


##### 7.2 python何时执行装饰器
装饰器的一个关键特性是，它们在被装饰的函数定义之后立即执行。这通常是在导入时。

In [11]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('regitry ->', registry)
    f1()
    f2()
    f3()

main()

running register(<function f1 at 0x00000229A925AD08>)
running register(<function f2 at 0x00000229A925A510>)
running main()
regitry -> [<function f1 at 0x00000229A925AD08>, <function f2 at 0x00000229A925A510>]
running f1()
running f2()
running f3()


##### 7.3 使用装饰器改进“策略”模式

In [13]:
# proms列表中的值使用promtion装饰器填充
promos = []

def promtion(promo_func):
    promos.append(promo_func)
    return promo_func

@promtion
def fidelity(order):
    '''为积分为1000或以上的顾客提供5%折扣'''
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promtion
def bulk_item(order):
    '''单个商品为20个或以上时提供10折扣'''
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promtion
def large_order(order):
    '''订单中的不同商品达到10个或以上时提供7%折扣'''
    discount_items = {item.product for item in order.cart}
    if len(discount_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    '''选择可用的最佳折扣'''
    return max(promo(order) for promo in promos)

##### 7.4 变量作用域规则

In [21]:
b = 4
def f1(a):
    global b
    print(a)
    print(b)
f1(3)
print(b)

3
4
4


In [25]:
b = 4
def f2(a):
    print(a)
    print(b)
    b = 9
f1(3)
print(b)

3
4
4


In [26]:
from dis import dis
print(dis(f1))
print('----------')
print(dis(f2))

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE
None
----------
  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE
None


编译器把 b 视作局部变量，即使在后
面才为 b 赋值，因为变量的种类（是不是局部变量）不能改变函数
的定义体。

##### 7.5 闭包
闭包指延伸了作用域的函数，其中包含函数定义体中引用、但是
不在定义体中定义的非全局变量。函数是不是匿名的没有关系，关键是
它能访问定义体之外定义的非全局变量。  
闭包是一种函数，它会保留定义函数时存在的自由变量的绑定，
这样调用函数时，虽然定义作用域不可用了，但是仍能使用那些绑定。

In [4]:
# 计算移动平均值的高阶函数
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)

    return averager

In [5]:
avg = make_averager()
print(avg(10), avg(11), avg(12))

10.0 10.5 11.0


In [13]:
print(avg.__code__.co_varnames, avg.__code__.co_freevars)
print(avg.__closure__)
print(avg.__closure__[0].cell_contents)

('new_value', 'total') ('series',)
(<cell at 0x00000138256F66A8: list object at 0x00000138259D7788>,)
[10, 11, 12]


##### 7.6 nonlocal 声明

In [17]:
# 计算移动平均值的高阶函数，不保存所有历史值，但有缺陷
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

In [18]:
avg = make_averager()
print(avg(10))

10.0


##### 7.7 实现一个简单的装饰器

In [13]:
# 一个简单的装饰器，它会在每次调用被装饰的函数时计时，然后把经过的时间、传入的参数和调用的结果打印出来。
import time
from functools import wraps

def clock(func):
    @wraps(func)
    def clocked(*args, **kw):
        t0 = time.perf_counter()
        result = func(*args, **kw)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kw:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kw.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [14]:
# 使用clock装饰器
@clock
def snooze(seconds):
    time.sleep(seconds)


@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

In [15]:
print('*' * 40, 'Calling snooze(.123)')
print(snooze(.123))
print('*' * 40, 'Calling factorial(6)')
print(factorial(6))

**************************************** Calling snooze(.123)
[0.12322828s] snooze(0.123) -> None
None
**************************************** Calling factorial(6)
[0.00000045s] factorial(1) -> 1
[0.00113570s] factorial(2) -> 2
[0.00183319s] factorial(3) -> 6
[0.00249810s] factorial(4) -> 24
[0.00357400s] factorial(5) -> 120
[0.00384398s] factorial(6) -> 720
720


##### 7.8 标准库中的装饰器

In [18]:
# 生成第N个斐波那契数，递归方式非常耗时
from functools import lru_cache

@lru_cache()
@clock
def fibonacii(n):
    if n < 2:
        return n
    return fibonacii(n-2) + fibonacii(n-1)

In [19]:
print(fibonacii(6))

[0.00000223s] fibonacii(0) -> 0
[0.00000134s] fibonacii(1) -> 1
[0.00069302s] fibonacii(2) -> 1
[0.00000491s] fibonacii(3) -> 2
[0.00099513s] fibonacii(4) -> 3
[0.00000178s] fibonacii(5) -> 5
[0.00107367s] fibonacii(6) -> 8
8


In [20]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [22]:
print(htmlize({1, 2, 3}))
print(htmlize('dfd'))

<pre>{1, 2, 3}</pre>
<pre>&#x27;dfd&#x27;</pre>


In [25]:
# singledispatch创建一个自定义的htmlize.register装饰器，把多个函数绑在一起组成一个泛函数
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br />\n')
    return '<p>{}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(text):
    return '<pre>{} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

##### 7.9 叠放装饰器
@d1  
@d2  
def f(): print('f')  
等于：  
def f(): print('f')  
f = d1(d2(f))  

##### 7.10 参数化装饰器
创建一个装饰器工厂函数，把参数传给它，返回一个装饰器，然后再把它应用到要装饰的函数上。

In [34]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x00000289EF2970D0>)
running main()
registry -> [<function f1 at 0x00000289EF2970D0>]
running f1()


In [51]:
# 为了接收参数，新的register装饰器必须作为函数调用
registry = set()


def register(active=True):
    def decorate(func):
        print('running register(active=%s) -> decorate(%s)'
              % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

In [52]:
@register(active=False)
def f1():
    print('running f1()')


@register()
def f2():
    print('running f2()')


def f3():
    print('runing f3()')

running register(active=False) -> decorate(<function f1 at 0x00000289EF721AE8>)
running register(active=True) -> decorate(<function f2 at 0x00000289EEF739D8>)


In [89]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return result
        return clocked
    return decorate

In [90]:
@clock()
def snooze(seconds):
    time.sleep(seconds)

In [91]:
for i in range(3):
    snooze(.123)

[0.12392879s] snooze(0.123) -> None
[0.12492585s] snooze(0.123) -> None
[0.12492537s] snooze(0.123) -> None
